In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn import preprocessing

raw_csv_data = np.loadtxt('tensorflow_project\\Audiobooks_data.csv', delimiter= ',')

unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]

In [3]:
#balance dataset

num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove= []

for i in range(targets_all.shape[0]):
    if targets_all[i] ==0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)
            
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis = 0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)



In [4]:
#standarize the inputs

scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)


In [5]:
#shuffle the data

shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]


In [6]:
#split the dataset into train,validation, and test

samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets/train_samples_count))
print(np.sum(validation_targets), validation_samples_count ,np.sum(validation_targets/validation_samples_count))
print(np.sum(test_targets), test_samples_count, np.sum(test_targets/test_samples_count))

1817.0 3579 0.5076837105336687
208.0 447 0.46532438478747196
212.0 448 0.47321428571428564


In [7]:
np.random.shuffle(shuffled_indices)

In [8]:
np.savez('Audiobooks_data_train',inputs = train_inputs, targets= train_targets)
np.savez('Audiobooks_data_validation',inputs = validation_inputs, targets= validation_targets)
np.savez('Audiobooks_data_test',inputs = test_inputs, targets= test_targets)

# Machine Learning Algorithm

In [9]:
npz = np.load('Audiobooks_data_train.npz')

train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_validation.npz')
valiation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_test.npz')
test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

In [10]:
#MODEL

input_size = 10
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
                            tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                            tf.keras.layers.Dense(output_size,activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size = 100

max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs,
          train_targets,
          batch_size = batch_size,
          epochs= max_epochs,
          callbacks=[early_stopping],
          validation_data=(valiation_inputs,validation_targets),
          verbose=2)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 3579 samples, validate on 447 samples
Epoch 1/100
3579/3579 - 1s - loss: 0.4964 - acc: 0.8206 - val_loss: 0.3635 - val_acc: 0.8680
Epoch 2/100
3579/3579 - 0s - loss: 0.3255 - acc: 0.8838 - val_loss: 0.3134 - val_acc: 0.8792
Epoch 3/100
3579/3579 - 0s - loss: 0.2867 - acc: 0.8938 - val_loss: 0.2815 - val_acc: 0.8904
Epoch 4/100
3579/3579 - 0s - loss: 0.2686 - acc: 0.9030 - val_loss: 0.2676 - val_acc: 0.8971
Epoch 5/100
3579/3579 - 0s - loss: 0.2584 - acc: 0.9033 - val_loss: 0.2648 - val_acc: 0.9016
Epoch 6/100
3579/3579 - 0s - loss: 0.2513 - acc: 0.9050 - val_loss: 0.2561 - val_acc: 0.9016
Epoch 7/100
3579/3579 - 0s - loss: 0.2470 - acc: 0.9084 - val_loss: 0.2533 - val_acc: 0.9038
Epoch 8/100
3579/3579 - 0s - loss: 0.2455 - acc: 0.9075 - val_loss: 0.2527 - val_acc: 0.9016
Epoch 9/100
3579/3579 - 0s - loss: 0.2443 - acc: 0.9103 - val_loss: 0.2505 - val_acc: 0.9016
Epoch 10/100
3579/3579 - 0s - loss:

In [11]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

448/448 [==============================] - 0s 25us/sample - loss: 0.2511 - acc: 0.9040


In [12]:
print('\n Test loss: {1:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


 Test loss: 90.40. Test accuracy: 90.40%
